
# DataLog - Solução de Previsão de Demanda com Machine Learning

- Este código implementa uma solução completa para previsão de demanda logística utilizando técnicas de aprendizado de máquina, conforme solicitado na Avaliação A3.

## Autores: Equipe DataLog Analytics

In [28]:
""" Importando bibliotecas necessárias """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import random
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from statsmodels.tsa.seasonal import seasonal_decompose
import warnings
import os

# Principais bibliotecas utilizadas
- pandas: manipulação e análise de dados em tabelas.
- numpy: operações matemáticas e manipulação eficiente de arrays.
- matplotlib.pyplot e seaborn: criação de gráficos e visualizações de dados.
- datetime e timedelta: manipulação de datas e tempos.
- random: geração de números aleatórios.
- scikit-learn: ferramentas para machine learning.
- warnings: controle de avisos do Python.
- os: manipulação de diretórios e arquivos do sistema operacional.

In [29]:
"""Cofigurações Iniciais"""

warnings.filterwarnings("ignore")
np.random.seed(42)
random.seed(42)
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("viridis")

""" Criação de diretório de trabalho """
os.makedirs("output", exist_ok=True)

# Início da Etapa 1 - Coleta e preparação de dados

In [30]:
""" Etapa 1 - Coleta e preparação de dados 
    - Gerando dados sintéticos para simular a demanda de produtos em diferentes centros de distribuição (CDs)
    - Definindo características dos CDs, eventos especiais e categorias de produtos
    - Definindo processos logísticos por CD
"""


CDS = {
    "Belem (PA)": {"lat": -1.4558, "lon": -48.5044, "capacidade_diaria": 1200},
    "Recife (PE)": {"lat": -8.0476, "lon": -34.8770, "capacidade_diaria": 1800},
    "Brasilia (DF)": {"lat": -15.7942, "lon": -47.8822, "capacidade_diaria": 2000},
    "Sao Paulo (SP)": {"lat": -23.5505, "lon": -46.6333, "capacidade_diaria": 5000},
    "Florianopolis (SC)": {"lat": -27.5954, "lon": -48.5480, "capacidade_diaria": 1500}
}

# Definição de eventos especiais que afetam a demanda
EVENTOS_ESPECIAIS = {
    "Black Friday": {"data_inicio": "2024-11-25", "data_fim": "2024-11-30", "impacto": 2.5},
    "Natal": {"data_inicio": "2024-12-15", "data_fim": "2024-12-24", "impacto": 2.0},
    "Dia das Mães": {"data_inicio": "2024-05-05", "data_fim": "2024-05-12", "impacto": 1.8},
    "Dia dos Pais": {"data_inicio": "2024-08-04", "data_fim": "2024-08-11", "impacto": 1.5},
    "Dia das Crianças": {"data_inicio": "2024-10-05", "data_fim": "2024-10-12", "impacto": 1.7},
    "Carnaval": {"data_inicio": "2024-02-10", "data_fim": "2024-02-14", "impacto": 0.7},  # Redução na demanda
    "Cyber Monday": {"data_inicio": "2024-12-02", "data_fim": "2024-12-02", "impacto": 2.0},
    "Volta às Aulas": {"data_inicio": "2024-01-15", "data_fim": "2024-02-05", "impacto": 1.6}
}

# Definição de categorias de produtos
CATEGORIAS_PRODUTOS = {
    "Eletrônicos": {"peso_medio": 3.5, "volume_medio": 0.015, "valor_medio": 1200, "sazonalidade": "alta",
                    "complexidade_picking": "alta", "fragilidade": "alta", "tempo_base_picking": 4.5,
                    "tempo_base_embalagem": 3.5, "prioridade_expedicao": "media"},
    "Vestuário": {"peso_medio": 0.8, "volume_medio": 0.005, "valor_medio": 150, "sazonalidade": "media",
                 "complexidade_picking": "media", "fragilidade": "baixa", "tempo_base_picking": 2.5,
                 "tempo_base_embalagem": 1.5, "prioridade_expedicao": "baixa"},
    "Alimentos": {"peso_medio": 5.0, "volume_medio": 0.02, "valor_medio": 80, "sazonalidade": "baixa",
                 "complexidade_picking": "baixa", "fragilidade": "media", "tempo_base_picking": 2.0,
                 "tempo_base_embalagem": 1.8, "prioridade_expedicao": "alta"},
    "Casa e Decoração": {"peso_medio": 8.0, "volume_medio": 0.05, "valor_medio": 300, "sazonalidade": "media",
                        "complexidade_picking": "alta", "fragilidade": "alta", "tempo_base_picking": 5.0,
                        "tempo_base_embalagem": 4.0, "prioridade_expedicao": "baixa"},
    "Beleza e Saúde": {"peso_medio": 1.2, "volume_medio": 0.008, "valor_medio": 120, "sazonalidade": "baixa",
                      "complexidade_picking": "media", "fragilidade": "media", "tempo_base_picking": 3.0,
                      "tempo_base_embalagem": 2.0, "prioridade_expedicao": "media"},
    "Esportes": {"peso_medio": 4.0, "volume_medio": 0.025, "valor_medio": 250, "sazonalidade": "alta",
                "complexidade_picking": "alta", "fragilidade": "media", "tempo_base_picking": 3.5,
                "tempo_base_embalagem": 2.5, "prioridade_expedicao": "baixa"},
    "Livros e Mídia": {"peso_medio": 1.0, "volume_medio": 0.004, "valor_medio": 70, "sazonalidade": "media",
                      "complexidade_picking": "baixa", "fragilidade": "baixa", "tempo_base_picking": 1.5,
                      "tempo_base_embalagem": 1.0, "prioridade_expedicao": "baixa"},
    "Brinquedos": {"peso_medio": 2.0, "volume_medio": 0.018, "valor_medio": 100, "sazonalidade": "alta",
                  "complexidade_picking": "media", "fragilidade": "media", "tempo_base_picking": 3.0,
                  "tempo_base_embalagem": 2.5, "prioridade_expedicao": "media"}
}

# Definição de características dos processos logísticos por CD
PROCESSOS_LOGISTICOS_CD = {
    "Belem (PA)": {
        "eficiencia_picking": 0.85,  # Eficiência relativa do processo de picking (1.0 = 100%)
        "eficiencia_embalagem": 0.80,  # Eficiência relativa do processo de embalagem
        "eficiencia_expedicao": 0.75,  # Eficiência relativa do processo de expedição
        "taxa_erro_picking_base": 0.05,  # Taxa base de erro no picking
        "taxa_erro_embalagem_base": 0.03,  # Taxa base de erro na embalagem
        "tempo_medio_expedicao": 120,  # Tempo médio de expedição em minutos
        "nivel_automacao": "baixo",  # Nível de automação do CD
        "capacidade_picking_hora": 80,  # Itens que podem ser separados por hora
        "capacidade_embalagem_hora": 60,  # Itens que podem ser embalados por hora
        "capacidade_expedicao_hora": 40  # Pedidos que podem ser expedidos por hora
    },
    "Recife (PE)": {
        "eficiencia_picking": 0.90,
        "eficiencia_embalagem": 0.85,
        "eficiencia_expedicao": 0.80,
        "taxa_erro_picking_base": 0.04,
        "taxa_erro_embalagem_base": 0.025,
        "tempo_medio_expedicao": 100,
        "nivel_automacao": "medio",
        "capacidade_picking_hora": 100,
        "capacidade_embalagem_hora": 80,
        "capacidade_expedicao_hora": 50
    },
    "Brasilia (DF)": {
        "eficiencia_picking": 0.92,
        "eficiencia_embalagem": 0.88,
        "eficiencia_expedicao": 0.85,
        "taxa_erro_picking_base": 0.035,
        "taxa_erro_embalagem_base": 0.02,
        "tempo_medio_expedicao": 90,
        "nivel_automacao": "medio",
        "capacidade_picking_hora": 120,
        "capacidade_embalagem_hora": 90,
        "capacidade_expedicao_hora": 60
    },
    "Sao Paulo (SP)": {
        "eficiencia_picking": 0.95,
        "eficiencia_embalagem": 0.93,
        "eficiencia_expedicao": 0.90,
        "taxa_erro_picking_base": 0.025,
        "taxa_erro_embalagem_base": 0.015,
        "tempo_medio_expedicao": 60,
        "nivel_automacao": "alto",
        "capacidade_picking_hora": 200,
        "capacidade_embalagem_hora": 150,
        "capacidade_expedicao_hora": 100
    },
    "Florianopolis (SC)": {
        "eficiencia_picking": 0.88,
        "eficiencia_embalagem": 0.86,
        "eficiencia_expedicao": 0.82,
        "taxa_erro_picking_base": 0.03,
        "taxa_erro_embalagem_base": 0.02,
        "tempo_medio_expedicao": 80,
        "nivel_automacao": "medio",
        "capacidade_picking_hora": 110,
        "capacidade_embalagem_hora": 85,
        "capacidade_expedicao_hora": 55
    }
}


In [31]:
"""Etapa 1 - Coleta e preparação de dados
- Validação da estrutura de dados
- geração de dados sintéticos
"""

# Função para verificar se uma data está dentro de um evento especial
def verificar_evento_especial(data):
    """
    Verifica se uma data específica está dentro de algum evento especial.

    Args:
        data: Data a ser verificada

    Returns:
        Tupla com nome do evento e fator de impacto, ou (None, 1.0) se não houver evento
    """
    for evento, info in EVENTOS_ESPECIAIS.items():
        data_inicio = pd.to_datetime(info["data_inicio"])
        data_fim = pd.to_datetime(info["data_fim"])

        if data_inicio <= data <= data_fim:
            return evento, info["impacto"]

    return None, 1.0


# Função para gerar dados sintéticos de demanda logística
def gerar_dados_demanda(dias=365, pedidos_base=50000):
    """
    Gera dados sintéticos de demanda logística para análise e modelagem.

    Args:
        dias: Número de dias para gerar dados
        pedidos_base: Número base de pedidos diários

    Returns:
        DataFrame com dados de demanda
    """
    print("Gerando dados sintéticos de demanda logística...")

    # Data inicial e final
    data_inicio = pd.to_datetime("2024-01-01")
    datas = [data_inicio + timedelta(days=i) for i in range(dias)]

    # Criar DataFrame base
    dados = []

    # Gerar dados diários
    for data in datas:
        # Verificar se é fim de semana (redução na demanda)
        fator_dia_semana = 0.7 if data.weekday() >= 5 else 1.0

        # Verificar eventos especiais
        evento, fator_evento = verificar_evento_especial(data)

        # Fator de sazonalidade mensal (maior demanda no fim do ano)
        fator_mes = 1.0 + (data.month / 12) * 0.5

        # Tendência de crescimento ao longo do ano
        fator_tendencia = 1.0 + (data - data_inicio).days / (365 * 2)

        # Para cada centro de distribuição
        for cd_nome, cd_info in CDS.items():
            # Obter informações de processos logísticos para este CD
            processos_cd = PROCESSOS_LOGISTICOS_CD[cd_nome]

            # Calcular demanda base para este CD (proporcional à capacidade)
            demanda_base_cd = pedidos_base * (cd_info["capacidade_diaria"] / 5000)

            # Aplicar fatores
            demanda_base = (
                demanda_base_cd
                * fator_dia_semana
                * fator_evento
                * fator_mes
                * fator_tendencia
            )

            # Adicionar ruído aleatório (±15%)
            ruido = np.random.uniform(0.85, 1.15)
            demanda = int(demanda_base * ruido)

            # Para cada categoria de produto
            for categoria, cat_info in CATEGORIAS_PRODUTOS.items():
                # Fator de sazonalidade por categoria
                fator_sazonalidade = 1.0
                if cat_info["sazonalidade"] == "alta":
                    # Produtos com alta sazonalidade são mais afetados por eventos e meses
                    fator_sazonalidade = fator_evento * fator_mes
                elif cat_info["sazonalidade"] == "media":
                    # Produtos com média sazonalidade são moderadamente afetados
                    fator_sazonalidade = 1.0 + (fator_evento - 1.0) * 0.7

                # Calcular proporção desta categoria no total (diferente por CD)
                if cd_nome == "Sao Paulo (SP)":
                    # São Paulo tem mais eletrônicos e vestuário
                    proporcao = {
                        "Eletrônicos": 0.25,
                        "Vestuário": 0.20,
                        "Alimentos": 0.10,
                        "Casa e Decoração": 0.15,
                        "Beleza e Saúde": 0.10,
                        "Esportes": 0.08,
                        "Livros e Mídia": 0.07,
                        "Brinquedos": 0.05,
                    }
                elif cd_nome == "Recife (PE)" or cd_nome == "Belem (PA)":
                    # Recife e Belém têm mais alimentos e beleza
                    proporcao = {
                        "Eletrônicos": 0.15,
                        "Vestuário": 0.15,
                        "Alimentos": 0.20,
                        "Casa e Decoração": 0.10,
                        "Beleza e Saúde": 0.15,
                        "Esportes": 0.10,
                        "Livros e Mídia": 0.05,
                        "Brinquedos": 0.10,
                    }
                else:
                    # Outros CDs têm distribuição mais equilibrada
                    proporcao = {
                        "Eletrônicos": 0.18,
                        "Vestuário": 0.18,
                        "Alimentos": 0.15,
                        "Casa e Decoração": 0.12,
                        "Beleza e Saúde": 0.12,
                        "Esportes": 0.10,
                        "Livros e Mídia": 0.08,
                        "Brinquedos": 0.07,
                    }

                # Calcular demanda para esta categoria
                demanda_categoria = int(
                    demanda * proporcao[categoria] * fator_sazonalidade
                )

                # Adicionar ruído específico da categoria (±10%)
                ruido_categoria = np.random.uniform(0.9, 1.1)
                demanda_categoria = int(demanda_categoria * ruido_categoria)

                # Calcular métricas derivadas
                peso_total = demanda_categoria * cat_info["peso_medio"]
                volume_total = demanda_categoria * cat_info["volume_medio"]
                valor_total = demanda_categoria * cat_info["valor_medio"]

                # Calcular capacidade utilizada (%)
                capacidade_utilizada = (
                    demanda_categoria / cd_info["capacidade_diaria"]
                ) * 100

                # Calcular lead time (dias) - varia por CD e categoria
                lead_time_base = {
                    "Belem (PA)": 3.5,
                    "Recife (PE)": 3.0,
                    "Brasilia (DF)": 2.5,
                    "Sao Paulo (SP)": 1.5,
                    "Florianopolis (SC)": 2.0,
                }

                # Ajuste de lead time por categoria
                ajuste_lead_time = {
                    "Eletrônicos": 1.2,
                    "Vestuário": 1.0,
                    "Alimentos": 0.8,
                    "Casa e Decoração": 1.5,
                    "Beleza e Saúde": 0.9,
                    "Esportes": 1.1,
                    "Livros e Mídia": 0.7,
                    "Brinquedos": 1.0,
                }

                # Lead time final
                lead_time = lead_time_base[cd_nome] * ajuste_lead_time[categoria]

                # Adicionar ruído ao lead time (±20%)
                lead_time = lead_time * np.random.uniform(0.8, 1.2)

                # Calcular taxa de atraso (%) - maior quando a demanda está próxima da capacidade
                taxa_atraso_base = (
                    max(0, min(50, (capacidade_utilizada - 70) * 1.5))
                    if capacidade_utilizada > 70
                    else 5
                )
                taxa_atraso = taxa_atraso_base * np.random.uniform(0.8, 1.2)

                # Calcular métricas de picking
                # Tempo base de picking ajustado pela eficiência do CD e complexidade do produto
                eficiencia_picking = processos_cd["eficiencia_picking"]
                complexidade_picking = 1.0
                if cat_info["complexidade_picking"] == "alta":
                    complexidade_picking = 1.3
                elif cat_info["complexidade_picking"] == "media":
                    complexidade_picking = 1.0
                else:  # baixa
                    complexidade_picking = 0.8

                # Tempo de picking por item (minutos)
                tempo_picking_item = (
                    cat_info["tempo_base_picking"]
                    * complexidade_picking
                    / eficiencia_picking
                )

                # Tempo total de picking para esta categoria/demanda
                tempo_total_picking = tempo_picking_item * demanda_categoria

                # Taxa de erro de picking - afetada pela complexidade e eficiência
                taxa_erro_picking_base = processos_cd["taxa_erro_picking_base"]
                taxa_erro_picking = (
                    taxa_erro_picking_base * complexidade_picking / eficiencia_picking
                )

                # Adicionar variação com base na ocupação do CD
                if capacidade_utilizada > 85:
                    # Mais erros quando o CD está muito ocupado
                    taxa_erro_picking *= 1.5

                # Calcular métricas de embalagem
                # Tempo base de embalagem ajustado pela eficiência do CD e fragilidade do produto
                eficiencia_embalagem = processos_cd["eficiencia_embalagem"]
                fragilidade = 1.0
                if cat_info["fragilidade"] == "alta":
                    fragilidade = 1.4
                elif cat_info["fragilidade"] == "media":
                    fragilidade = 1.1
                else:  # baixa
                    fragilidade = 0.9

                # Tempo de embalagem por item (minutos)
                tempo_embalagem_item = (
                    cat_info["tempo_base_embalagem"]
                    * fragilidade
                    / eficiencia_embalagem
                )

                # Tempo total de embalagem para esta categoria/demanda
                tempo_total_embalagem = tempo_embalagem_item * demanda_categoria

                # Taxa de erro de embalagem - afetada pela fragilidade e eficiência
                taxa_erro_embalagem_base = processos_cd["taxa_erro_embalagem_base"]
                taxa_erro_embalagem = (
                    taxa_erro_embalagem_base * fragilidade / eficiencia_embalagem
                )

                # Adicionar variação com base na ocupação do CD
                if capacidade_utilizada > 85:
                    # Mais erros quando o CD está muito ocupado
                    taxa_erro_embalagem *= 1.4

                # Calcular métricas de expedição
                # Tempo de expedição ajustado pela eficiência do CD e prioridade do produto
                eficiencia_expedicao = processos_cd["eficiencia_expedicao"]
                prioridade = 1.0
                if cat_info["prioridade_expedicao"] == "alta":
                    prioridade = 0.8  # Produtos de alta prioridade são expedidos mais rapidamente
                elif cat_info["prioridade_expedicao"] == "media":
                    prioridade = 1.0
                else:  # baixa
                    prioridade = 1.2

                # Tempo de expedição por pedido (minutos)
                tempo_expedicao = (
                    processos_cd["tempo_medio_expedicao"]
                    * prioridade
                    / eficiencia_expedicao
                )

                # Adicionar variação com base na ocupação do CD
                if capacidade_utilizada > 90:
                    # Expedição mais lenta quando o CD está muito ocupado
                    tempo_expedicao *= 1.5

                # Calcular gargalos nos processos
                # Capacidade diária de picking (itens)
                capacidade_picking_dia = (
                    processos_cd["capacidade_picking_hora"] * 16
                )  # Considerando 16h de operação
                gargalo_picking = (demanda_categoria / capacidade_picking_dia) * 100

                # Capacidade diária de embalagem (itens)
                capacidade_embalagem_dia = (
                    processos_cd["capacidade_embalagem_hora"] * 16
                )
                gargalo_embalagem = (demanda_categoria / capacidade_embalagem_dia) * 100

                # Capacidade diária de expedição (pedidos)
                capacidade_expedicao_dia = (
                    processos_cd["capacidade_expedicao_hora"] * 16
                )
                # Assumindo que cada pedido tem em média 3 itens
                pedidos_estimados = demanda_categoria / 3
                gargalo_expedicao = (pedidos_estimados / capacidade_expedicao_dia) * 100

                # Adicionar registro ao dataset
                dados.append(
                    {
                        "data": data,
                        "centro_distribuicao": cd_nome,
                        "categoria_produto": categoria,
                        "demanda": demanda_categoria,
                        "peso_total_kg": round(peso_total, 2),
                        "volume_total_m3": round(volume_total, 3),
                        "valor_total": round(valor_total, 2),
                        "capacidade_utilizada_pct": round(capacidade_utilizada, 2),
                        "lead_time_dias": round(lead_time, 2),
                        "taxa_atraso_pct": round(taxa_atraso, 2),
                        "evento_especial": evento,
                        "dia_semana": data.day_name(),
                        "mes": data.month_name(),
                        "trimestre": f"Q{(data.month-1)//3 + 1}",
                        "tempo_picking_item_min": round(tempo_picking_item, 2),
                        "tempo_total_picking_min": round(tempo_total_picking, 2),
                        "taxa_erro_picking_pct": round(taxa_erro_picking * 100, 2),
                        "gargalo_picking_pct": round(gargalo_picking, 2),
                        "complexidade_picking": cat_info["complexidade_picking"],
                        "tempo_embalagem_item_min": round(tempo_embalagem_item, 2),
                        "tempo_total_embalagem_min": round(tempo_total_embalagem, 2),
                        "taxa_erro_embalagem_pct": round(taxa_erro_embalagem * 100, 2),
                        "gargalo_embalagem_pct": round(gargalo_embalagem, 2),
                        "fragilidade_produto": cat_info["fragilidade"],
                        "tempo_expedicao_min": round(tempo_expedicao, 2),
                        "gargalo_expedicao_pct": round(gargalo_expedicao, 2),
                        "prioridade_expedicao": cat_info["prioridade_expedicao"],
                        "nivel_automacao_cd": processos_cd["nivel_automacao"],
                        "tempo_total_processamento_min": round(
                            tempo_picking_item + tempo_embalagem_item + tempo_expedicao,
                            2,
                        ),
                    }
                )

    # Criar DataFrame
    df = pd.DataFrame(dados)

    # Adicionar features temporais
    df["dia_do_mes"] = df["data"].dt.day
    df["dia_do_ano"] = df["data"].dt.dayofyear
    df["semana_do_ano"] = df["data"].dt.isocalendar().week
    df["fim_de_semana"] = df["data"].dt.weekday >= 5

    print(f"Dados gerados com sucesso: {len(df)} registros.")
    return df

In [32]:
""" Etapa 1 - Coleta e preparação de dados
- Limpeza e preparação dos dados
"""

# Função para preparar os dados para modelagem
def preparar_dados_modelagem(df):
    """
    Prepara os dados para modelagem, incluindo engenharia de features,
    tratamento de valores ausentes e normalização.

    Args:
        df: DataFrame com dados brutos

    Returns:
        DataFrame com dados preparados para modelagem
    """
    print("Preparando dados para modelagem...")

    # Criar cópia para não modificar o original
    df_prep = df.copy()

    # Converter colunas categóricas para numéricas
    df_prep["mes_num"] = df_prep["data"].dt.month
    df_prep["dia_semana_num"] = df_prep["data"].dt.weekday

    # Criar flag para eventos especiais (1 se houver evento, 0 caso contrário)
    df_prep["tem_evento"] = df_prep["evento_especial"].notna().astype(int)

    # Criar features de lag (demanda dos dias anteriores)
    # Inicializar colunas de lag e médias móveis com NaN
    for lag in [1, 7, 14]:
        df_prep[f"demanda_lag_{lag}"] = np.nan
    df_prep["demanda_ma7"] = np.nan
    df_prep["demanda_ma14"] = np.nan

    for cd in df_prep["centro_distribuicao"].unique():
        for categoria in df_prep["categoria_produto"].unique():
            # Filtrar dados para este CD e categoria
            mask = (df_prep["centro_distribuicao"] == cd) & (
                df_prep["categoria_produto"] == categoria
            )

            # Ordenar por data
            temp = df_prep.loc[mask].sort_values("data")

            # Criar lags de 1, 7 e 14 dias
            for lag in [1, 7, 14]:
                col_name = f"demanda_lag_{lag}"
                temp[col_name] = temp["demanda"].shift(lag)

            # Calcular média móvel de 7 e 14 dias
            temp["demanda_ma7"] = temp["demanda"].rolling(window=7).mean()
            temp["demanda_ma14"] = temp["demanda"].rolling(window=14).mean()

            # Atualizar no DataFrame principal
            df_prep.loc[
                mask,
                [f"demanda_lag_{lag}" for lag in [1, 7, 14]]
                + ["demanda_ma7", "demanda_ma14"],
            ] = temp[
                [f"demanda_lag_{lag}" for lag in [1, 7, 14]]
                + ["demanda_ma7", "demanda_ma14"]
            ]

    # Identificar colunas numéricas de lag e médias móveis
    colunas_lag = [f"demanda_lag_{lag}" for lag in [1, 7, 14]] + [
        "demanda_ma7",
        "demanda_ma14",
    ]

    # Preencher valores NaN nas colunas de lag com a média
    for col in colunas_lag:
        if col in df_prep.columns and pd.api.types.is_numeric_dtype(df_prep[col]):
            df_prep[col] = df_prep[col].fillna(df_prep[col].mean())
        else:
            print(
                f"Aviso: Coluna '{col}' não encontrada ou não numérica. Ignorando fillna."
            )

    # Criar features de tendência
    df_prep["dias_desde_inicio"] = (df_prep["data"] - df_prep["data"].min()).dt.days

    # Criar features cíclicas para dia do ano e semana do ano
    df_prep["dia_ano_sin"] = np.sin(2 * np.pi * df_prep["dia_do_ano"] / 365)
    df_prep["dia_ano_cos"] = np.cos(2 * np.pi * df_prep["dia_do_ano"] / 365)
    df_prep["semana_sin"] = np.sin(
        2 * np.pi * df_prep["semana_do_ano"].astype(float) / 52
    )
    df_prep["semana_cos"] = np.cos(
        2 * np.pi * df_prep["semana_do_ano"].astype(float) / 52
    )

    # Criar feature de proximidade com fim do mês (maior demanda próximo ao fim do mês)
    df_prep["prox_fim_mes"] = 1 - (df_prep["dia_do_mes"] / 31)

    # Codificar variáveis categóricas
    # Complexidade de picking
    df_prep["complexidade_picking_num"] = df_prep["complexidade_picking"].map(
        {"baixa": 0, "media": 1, "alta": 2}
    )

    # Fragilidade do produto
    df_prep["fragilidade_produto_num"] = df_prep["fragilidade_produto"].map(
        {"baixa": 0, "media": 1, "alta": 2}
    )

    # Prioridade de expedição
    df_prep["prioridade_expedicao_num"] = df_prep["prioridade_expedicao"].map(
        {"baixa": 0, "media": 1, "alta": 2}
    )

    # Nível de automação do CD
    df_prep["nivel_automacao_cd_num"] = df_prep["nivel_automacao_cd"].map(
        {"baixo": 0, "medio": 1, "alto": 2}
    )

    # Criar features de gargalos combinados
    df_prep["gargalo_combinado"] = (
        df_prep["gargalo_picking_pct"]
        + df_prep["gargalo_embalagem_pct"]
        + df_prep["gargalo_expedicao_pct"]
    ) / 3

    # Criar feature de eficiência do processo
    df_prep["eficiencia_processo"] = (
        100
        - (df_prep["taxa_erro_picking_pct"] + df_prep["taxa_erro_embalagem_pct"]) / 2
    )

    print("Dados preparados com sucesso.")
    return df_prep

# Fim da Etapa 1 - Coleta e preparação de dados
---
## Relação de Dados Gerados 

| Nome da Coluna                  | Descrição                                                                 | Exemplo                  |
|----------------------------------|--------------------------------------------------------------------------|--------------------------|
| data                            | Data da operação                                                         | 2024-05-10               |
| centro_distribuicao             | Nome do Centro de Distribuição                                           | Sao Paulo (SP)           |
| categoria_produto               | Categoria do produto                                                     | Eletrônicos              |
| demanda                         | Quantidade demandada para a categoria no CD                              | 320                      |
| peso_total_kg                   | Peso total dos produtos demandados (kg)                                  | 1120.50                  |
| volume_total_m3                 | Volume total dos produtos demandados (m³)                                | 4.800                    |
| valor_total                     | Valor total dos produtos demandados (R$)                                 | 384000.00                |
| capacidade_utilizada_pct        | Percentual da capacidade diária do CD utilizada                          | 64.00                    |
| lead_time_dias                  | Tempo médio de atendimento do pedido (dias)                              | 1.80                     |
| taxa_atraso_pct                 | Percentual de pedidos com atraso                                         | 7.20                     |
| evento_especial                 | Evento especial que impactou a demanda                                   | Black Friday             |
| dia_semana                      | Dia da semana                                                            | Friday                   |
| mes                             | Nome do mês                                                              | May                      |
| trimestre                       | Trimestre do ano                                                         | Q2                       |
| tempo_picking_item_min          | Tempo médio de picking por item (minutos)                                | 5.20                     |
| tempo_total_picking_min         | Tempo total de picking para a categoria (minutos)                        | 1664.00                  |
| taxa_erro_picking_pct           | Percentual de erros no picking                                           | 4.50                     |
| gargalo_picking_pct             | Percentual de ocupação da capacidade de picking                          | 45.00                    |
| complexidade_picking            | Complexidade do picking para a categoria                                 | alta                     |
| tempo_embalagem_item_min        | Tempo médio de embalagem por item (minutos)                              | 4.90                     |
| tempo_total_embalagem_min       | Tempo total de embalagem para a categoria (minutos)                      | 1568.00                  |
| taxa_erro_embalagem_pct         | Percentual de erros na embalagem                                         | 2.80                     |
| gargalo_embalagem_pct           | Percentual de ocupação da capacidade de embalagem                        | 39.00                    |
| fragilidade_produto             | Fragilidade do produto                                                   | alta                     |
| tempo_expedicao_min             | Tempo médio de expedição por pedido (minutos)                            | 75.00                    |
| gargalo_expedicao_pct           | Percentual de ocupação da capacidade de expedição                        | 30.00                    |
| prioridade_expedicao            | Prioridade de expedição da categoria                                     | media                    |
| nivel_automacao_cd              | Nível de automação do CD                                                 | alto                     |
| tempo_total_processamento_min   | Tempo total de processamento (picking + embalagem + expedição, minutos)  | 185.10                   |


# Início da Etapa 2 - Análise Exploratória de Dados

In [33]:
"""Etapa 2 - Análise exploratória e visualização
- Análise de correlação
- Visualização de dados
- Identificação de padrões e tendências
- Insights
"""


# Função para realizar análise exploratória dos dados
def analise_exploratoria(df):
    """
    Realiza análise exploratória dos dados, gerando visualizações e insights.

    Args:
        df: DataFrame com dados de demanda

    Returns:
        Dicionário com insights da análise
    """
    print("Realizando análise exploratória dos dados...")

    # Criar diretório para visualizações
    os.makedirs("output/visualizacoes", exist_ok=True)

    # Resumo estatístico
    print("\nResumo estatístico dos dados:")
    display(df.describe())

    # Verificar valores ausentes
    print("\nValores ausentes por coluna:")
    display(df.isna().sum())

    # Análise temporal da demanda
    plt.figure(figsize=(14, 7))
    demanda_diaria = df.groupby("data")["demanda"].sum().reset_index()
    plt.plot(demanda_diaria["data"], demanda_diaria["demanda"])
    plt.title("Demanda Total Diária")
    plt.xlabel("Data")
    plt.ylabel("Demanda (unidades)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/demanda_diaria.png")
    plt.close()

    # Demanda por centro de distribuição
    plt.figure(figsize=(14, 7))
    demanda_cd = (
        df.groupby(["data", "centro_distribuicao"])["demanda"].sum().reset_index()
    )
    for cd in demanda_cd["centro_distribuicao"].unique():
        data_cd = demanda_cd[demanda_cd["centro_distribuicao"] == cd]
        plt.plot(data_cd["data"], data_cd["demanda"], label=cd)
    plt.title("Demanda Diária por Centro de Distribuição")
    plt.xlabel("Data")
    plt.ylabel("Demanda (unidades)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/demanda_por_cd.png")
    plt.close()

    # Demanda por categoria de produto
    plt.figure(figsize=(14, 7))
    demanda_categoria = (
        df.groupby(["data", "categoria_produto"])["demanda"].sum().reset_index()
    )
    for categoria in demanda_categoria["categoria_produto"].unique():
        data_cat = demanda_categoria[
            demanda_categoria["categoria_produto"] == categoria
        ]
        plt.plot(data_cat["data"], data_cat["demanda"], label=categoria)
    plt.title("Demanda Diária por Categoria de Produto")
    plt.xlabel("Data")
    plt.ylabel("Demanda (unidades)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/demanda_por_categoria.png")
    plt.close()

    # Distribuição da demanda
    plt.figure(figsize=(10, 6))
    sns.histplot(df["demanda"], kde=True)
    plt.title("Distribuição da Demanda")
    plt.xlabel("Demanda (unidades)")
    plt.ylabel("Frequência")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/distribuicao_demanda.png")
    plt.close()

    # Boxplot da demanda por centro de distribuição
    plt.figure(figsize=(12, 6))
    sns.boxplot(x="centro_distribuicao", y="demanda", data=df)
    plt.title("Distribuição da Demanda por Centro de Distribuição")
    plt.xlabel("Centro de Distribuição")
    plt.ylabel("Demanda (unidades)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/boxplot_demanda_cd.png")
    plt.close()

    # Boxplot da demanda por categoria de produto
    plt.figure(figsize=(12, 6))
    sns.boxplot(x="categoria_produto", y="demanda", data=df)
    plt.title("Distribuição da Demanda por Categoria de Produto")
    plt.xlabel("Categoria de Produto")
    plt.ylabel("Demanda (unidades)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/boxplot_demanda_categoria.png")
    plt.close()

    # Heatmap da demanda por dia da semana e mês
    demanda_dia_mes = df.groupby(["dia_semana", "mes"])["demanda"].mean().reset_index()
    demanda_dia_mes_pivot = demanda_dia_mes.pivot(
        index="dia_semana", columns="mes", values="demanda"
    )

    # Reordenar dias da semana e meses
    dias_ordem = [
        "Monday",
        "Tuesday",
        "Wednesday",
        "Thursday",
        "Friday",
        "Saturday",
        "Sunday",
    ]
    meses_ordem = [
        "January",
        "February",
        "March",
        "April",
        "May",
        "June",
        "July",
        "August",
        "September",
        "October",
        "November",
        "December",
    ]

    demanda_dia_mes_pivot = demanda_dia_mes_pivot.reindex(dias_ordem)
    demanda_dia_mes_pivot = demanda_dia_mes_pivot.reindex(columns=meses_ordem)

    plt.figure(figsize=(14, 8))
    sns.heatmap(demanda_dia_mes_pivot, annot=True, fmt=".0f", cmap="YlGnBu")
    plt.title("Demanda Média por Dia da Semana e Mês")
    plt.tight_layout()
    plt.savefig("output/visualizacoes/heatmap_demanda_dia_mes.png")
    plt.close()

    # Impacto dos eventos especiais na demanda
    plt.figure(figsize=(12, 6))
    demanda_evento = df.groupby("evento_especial")["demanda"].mean().reset_index()
    demanda_evento = demanda_evento.sort_values("demanda", ascending=False)
    sns.barplot(x="evento_especial", y="demanda", data=demanda_evento)
    plt.title("Demanda Média por Evento Especial")
    plt.xlabel("Evento Especial")
    plt.ylabel("Demanda Média (unidades)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/demanda_por_evento.png")
    plt.close()

    # Correlação entre variáveis
    plt.figure(figsize=(16, 14))
    colunas_numericas = df.select_dtypes(include=[np.number]).columns
    corr = df[colunas_numericas].corr()
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", vmin=-1, vmax=1)
    plt.title("Matriz de Correlação")
    plt.tight_layout()
    plt.savefig("output/visualizacoes/matriz_correlacao.png")
    plt.close()

    # Decomposição da série temporal
    plt.figure(figsize=(14, 10))
    demanda_diaria = df.groupby("data")["demanda"].sum().reset_index()
    demanda_diaria.set_index("data", inplace=True)

    # Decomposição da série temporal
    decomposicao = seasonal_decompose(demanda_diaria, model="additive", period=7)

    plt.subplot(4, 1, 1)
    plt.plot(demanda_diaria)
    plt.title("Demanda Original")
    plt.grid(True)

    plt.subplot(4, 1, 2)
    plt.plot(decomposicao.trend)
    plt.title("Tendência")
    plt.grid(True)

    plt.subplot(4, 1, 3)
    plt.plot(decomposicao.seasonal)
    plt.title("Sazonalidade")
    plt.grid(True)

    plt.subplot(4, 1, 4)
    plt.plot(decomposicao.resid)
    plt.title("Resíduos")
    plt.grid(True)

    plt.tight_layout()
    plt.savefig("output/visualizacoes/decomposicao_serie_temporal.png")
    plt.close()

    # Análise de capacidade utilizada
    plt.figure(figsize=(14, 7))
    capacidade_cd = (
        df.groupby(["data", "centro_distribuicao"])["capacidade_utilizada_pct"]
        .mean()
        .reset_index()
    )
    for cd in capacidade_cd["centro_distribuicao"].unique():
        data_cd = capacidade_cd[capacidade_cd["centro_distribuicao"] == cd]
        plt.plot(data_cd["data"], data_cd["capacidade_utilizada_pct"], label=cd)
    plt.title("Capacidade Utilizada por Centro de Distribuição")
    plt.xlabel("Data")
    plt.ylabel("Capacidade Utilizada (%)")
    plt.axhline(y=80, color="r", linestyle="--", label="Limite de Alerta (80%)")
    plt.axhline(y=100, color="darkred", linestyle="--", label="Capacidade Máxima")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/capacidade_utilizada.png")
    plt.close()

    # Relação entre capacidade utilizada e taxa de atraso
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        x="capacidade_utilizada_pct",
        y="taxa_atraso_pct",
        hue="centro_distribuicao",
        data=df,
        alpha=0.5,
    )
    plt.title("Relação entre Capacidade Utilizada e Taxa de Atraso")
    plt.xlabel("Capacidade Utilizada (%)")
    plt.ylabel("Taxa de Atraso (%)")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/capacidade_vs_atraso.png")
    plt.close()

    # Análise dos processos de picking, embalagem e expedição
    # Tempo médio de picking por categoria de produto
    plt.figure(figsize=(12, 6))
    tempo_picking = (
        df.groupby("categoria_produto")["tempo_picking_item_min"].mean().reset_index()
    )
    tempo_picking = tempo_picking.sort_values("tempo_picking_item_min", ascending=False)
    sns.barplot(x="categoria_produto", y="tempo_picking_item_min", data=tempo_picking)
    plt.title("Tempo Médio de Picking por Categoria de Produto")
    plt.xlabel("Categoria de Produto")
    plt.ylabel("Tempo (minutos)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/tempo_picking_categoria.png")
    plt.close()

    # Tempo médio de embalagem por categoria de produto
    plt.figure(figsize=(12, 6))
    tempo_embalagem = (
        df.groupby("categoria_produto")["tempo_embalagem_item_min"].mean().reset_index()
    )
    tempo_embalagem = tempo_embalagem.sort_values(
        "tempo_embalagem_item_min", ascending=False
    )
    sns.barplot(
        x="categoria_produto", y="tempo_embalagem_item_min", data=tempo_embalagem
    )
    plt.title("Tempo Médio de Embalagem por Categoria de Produto")
    plt.xlabel("Categoria de Produto")
    plt.ylabel("Tempo (minutos)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/tempo_embalagem_categoria.png")
    plt.close()

    # Tempo médio de expedição por centro de distribuição
    plt.figure(figsize=(12, 6))
    tempo_expedicao = (
        df.groupby("centro_distribuicao")["tempo_expedicao_min"].mean().reset_index()
    )
    tempo_expedicao = tempo_expedicao.sort_values(
        "tempo_expedicao_min", ascending=False
    )
    sns.barplot(x="centro_distribuicao", y="tempo_expedicao_min", data=tempo_expedicao)
    plt.title("Tempo Médio de Expedição por Centro de Distribuição")
    plt.xlabel("Centro de Distribuição")
    plt.ylabel("Tempo (minutos)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/tempo_expedicao_cd.png")
    plt.close()

    # Taxa de erro de picking por centro de distribuição
    plt.figure(figsize=(12, 6))
    erro_picking = (
        df.groupby("centro_distribuicao")["taxa_erro_picking_pct"].mean().reset_index()
    )
    erro_picking = erro_picking.sort_values("taxa_erro_picking_pct", ascending=False)
    sns.barplot(x="centro_distribuicao", y="taxa_erro_picking_pct", data=erro_picking)
    plt.title("Taxa de Erro de Picking por Centro de Distribuição")
    plt.xlabel("Centro de Distribuição")
    plt.ylabel("Taxa de Erro (%)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/erro_picking_cd.png")
    plt.close()

    # Taxa de erro de embalagem por centro de distribuição
    plt.figure(figsize=(12, 6))
    erro_embalagem = (
        df.groupby("centro_distribuicao")["taxa_erro_embalagem_pct"]
        .mean()
        .reset_index()
    )
    erro_embalagem = erro_embalagem.sort_values(
        "taxa_erro_embalagem_pct", ascending=False
    )
    sns.barplot(
        x="centro_distribuicao", y="taxa_erro_embalagem_pct", data=erro_embalagem
    )
    plt.title("Taxa de Erro de Embalagem por Centro de Distribuição")
    plt.xlabel("Centro de Distribuição")
    plt.ylabel("Taxa de Erro (%)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/erro_embalagem_cd.png")
    plt.close()

    # Análise de gargalos nos processos
    plt.figure(figsize=(14, 7))
    gargalos = (
        df.groupby(["data", "centro_distribuicao"])[
            ["gargalo_picking_pct", "gargalo_embalagem_pct", "gargalo_expedicao_pct"]
        ]
        .mean()
        .reset_index()
    )

    # Criar um gráfico para cada tipo de gargalo
    plt.figure(figsize=(14, 7))
    for cd in gargalos["centro_distribuicao"].unique():
        data_cd = gargalos[gargalos["centro_distribuicao"] == cd]
        plt.plot(
            data_cd["data"], data_cd["gargalo_picking_pct"], label=f"{cd} - Picking"
        )
    plt.title("Gargalos de Picking por Centro de Distribuição")
    plt.xlabel("Data")
    plt.ylabel("Utilização da Capacidade (%)")
    plt.axhline(y=80, color="r", linestyle="--", label="Limite de Alerta (80%)")
    plt.axhline(y=100, color="darkred", linestyle="--", label="Capacidade Máxima")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/gargalos_picking.png")
    plt.close()

    plt.figure(figsize=(14, 7))
    for cd in gargalos["centro_distribuicao"].unique():
        data_cd = gargalos[gargalos["centro_distribuicao"] == cd]
        plt.plot(
            data_cd["data"], data_cd["gargalo_embalagem_pct"], label=f"{cd} - Embalagem"
        )
    plt.title("Gargalos de Embalagem por Centro de Distribuição")
    plt.xlabel("Data")
    plt.ylabel("Utilização da Capacidade (%)")
    plt.axhline(y=80, color="r", linestyle="--", label="Limite de Alerta (80%)")
    plt.axhline(y=100, color="darkred", linestyle="--", label="Capacidade Máxima")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/gargalos_embalagem.png")
    plt.close()

    plt.figure(figsize=(14, 7))
    for cd in gargalos["centro_distribuicao"].unique():
        data_cd = gargalos[gargalos["centro_distribuicao"] == cd]
        plt.plot(
            data_cd["data"], data_cd["gargalo_expedicao_pct"], label=f"{cd} - Expedição"
        )
    plt.title("Gargalos de Expedição por Centro de Distribuição")
    plt.xlabel("Data")
    plt.ylabel("Utilização da Capacidade (%)")
    plt.axhline(y=80, color="r", linestyle="--", label="Limite de Alerta (80%)")
    plt.axhline(y=100, color="darkred", linestyle="--", label="Capacidade Máxima")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/gargalos_expedicao.png")
    plt.close()

    # Comparação dos tempos de processamento
    plt.figure(figsize=(14, 7))
    tempos_processamento = (
        df.groupby("centro_distribuicao")[
            [
                "tempo_picking_item_min",
                "tempo_embalagem_item_min",
                "tempo_expedicao_min",
            ]
        ]
        .mean()
        .reset_index()
    )

    # Converter para formato longo para facilitar a visualização
    tempos_processamento_long = pd.melt(
        tempos_processamento,
        id_vars=["centro_distribuicao"],
        value_vars=[
            "tempo_picking_item_min",
            "tempo_embalagem_item_min",
            "tempo_expedicao_min",
        ],
        var_name="Processo",
        value_name="Tempo (min)",
    )

    # Mapear nomes mais legíveis para os processos
    tempos_processamento_long["Processo"] = tempos_processamento_long["Processo"].map(
        {
            "tempo_picking_item_min": "Picking",
            "tempo_embalagem_item_min": "Embalagem",
            "tempo_expedicao_min": "Expedição",
        }
    )

    plt.figure(figsize=(12, 6))
    sns.barplot(
        x="centro_distribuicao",
        y="Tempo (min)",
        hue="Processo",
        data=tempos_processamento_long,
    )
    plt.title("Comparação dos Tempos de Processamento por Centro de Distribuição")
    plt.xlabel("Centro de Distribuição")
    plt.ylabel("Tempo Médio (minutos)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/visualizacoes/comparacao_tempos_processamento.png")
    plt.close()

    # Insights da análise
    insights = {
        "padroes_temporais": "Identificamos claros padrões sazonais na demanda, com picos durante eventos especiais como Black Friday e Natal. Também há variações semanais, com menor demanda nos fins de semana.",
        "diferencas_cds": "O CD de São Paulo apresenta a maior demanda e utilização de capacidade, seguido por Brasília. Belém e Florianópolis têm demanda mais baixa e estável.",
        "categorias_produtos": "Eletrônicos e Vestuário são as categorias com maior demanda e maior variabilidade sazonal, enquanto Alimentos apresenta demanda mais estável ao longo do ano.",
        "gargalos_capacidade": "Identificamos períodos críticos onde a capacidade utilizada ultrapassa 80%, especialmente em São Paulo durante eventos especiais, o que está diretamente correlacionado com aumento nas taxas de atraso.",
        "fatores_influencia": "Os principais fatores que influenciam a demanda são: eventos especiais (impacto de até 150%), sazonalidade mensal (maior no fim do ano), dia da semana (menor nos fins de semana) e categoria do produto.",
        "gargalos_processos": "O processo de picking é o principal gargalo em todos os CDs, especialmente para produtos de alta complexidade como Eletrônicos e Casa e Decoração. A expedição apresenta gargalos significativos durante eventos especiais.",
        "erros_processos": "As taxas de erro de picking e embalagem aumentam significativamente quando a capacidade utilizada ultrapassa 85%, especialmente em CDs com baixo nível de automação como Belém e Florianópolis.",
        "eficiencia_processos": "São Paulo apresenta a maior eficiência em todos os processos devido ao alto nível de automação, enquanto Belém apresenta os maiores tempos de processamento e taxas de erro.",
    }

    print("Análise exploratória concluída com sucesso.")
    return insights

# Fim da Etapa 2 - Análise Exploratória de Dados
---
## Insights e Conclusões da análise Exploratória

### Principais insights identificados:

- Padrões temporais: Identificamos claros padrões sazonais na demanda, com picos durante eventos especiais como Black Friday e Natal. Também há variações semanais, com menor demanda nos fins de semana.
- Diferenças entre CDs: O CD de São Paulo apresenta a maior demanda e utilização de capacidade, seguido por Brasília. Belém e Florianópolis têm demanda mais baixa e estável.
- Categorias de produtos: Eletrônicos e Vestuário são as categorias com maior demanda e maior variabilidade sazonal, enquanto Alimentos apresenta demanda mais estável ao longo do ano.
- Gargalos de capacidade: Identificamos períodos críticos onde a capacidade utilizada ultrapassa 80%, especialmente em São Paulo durante eventos especiais, o que está diretamente correlacionado com aumento nas taxas de atraso.
- Fatores de influência: Os principais fatores que influenciam a demanda são: eventos especiais (impacto de até 150%), sazonalidade mensal (maior no fim do ano), dia da semana (menor nos fins de semana) e categoria do produto.
- Gargalos nos processos: O processo de picking é o principal gargalo em todos os CDs, especialmente para produtos de alta complexidade como Eletrônicos e Casa e Decoração. A expedição apresenta gargalos significativos durante eventos especiais.
- Erros nos processos: As taxas de erro de picking e embalagem aumentam significativamente quando a capacidade utilizada ultrapassa 85%, especialmente em CDs com baixo nível de automação como Belém e Florianópolis.
- Eficiência dos processos: São Paulo apresenta a maior eficiência em todos os processos devido ao alto nível de automação, enquanto Belém apresenta os maiores tempos de processamento e taxas de erro.

# Início da Etapa 3 - Desenvolvimento dos modelos

In [34]:
"""Etapa 3 - Desenvolvimento dos modelos
- treinamento de modelos
"""

# Função para treinar modelos de previsão de demanda
def treinar_modelos_previsao(df, horizonte_previsao=30):
    """
    Treina diferentes modelos de machine learning para previsão de demanda.

    Args:
        df: DataFrame com dados preparados
        horizonte_previsao: Número de dias para prever no futuro

    Returns:
        Dicionário com modelos treinados e métricas de avaliação
    """
    print("Treinando modelos de previsão de demanda...")

    # Criar diretório para modelos
    os.makedirs('output/modelos', exist_ok=True)

    # Preparar features e target
    features = [
        'mes_num', 'dia_semana_num', 'tem_evento',
        'demanda_lag_1', 'demanda_lag_7', 'demanda_lag_14',
        'demanda_ma7', 'demanda_ma14', 'dias_desde_inicio',
        'dia_ano_sin', 'dia_ano_cos', 'semana_sin', 'semana_cos',
        'prox_fim_mes', 'capacidade_utilizada_pct',
        'complexidade_picking_num', 'fragilidade_produto_num',
        'prioridade_expedicao_num', 'nivel_automacao_cd_num',
        'tempo_picking_item_min', 'tempo_embalagem_item_min',
        'tempo_expedicao_min', 'gargalo_picking_pct',
        'gargalo_embalagem_pct', 'gargalo_expedicao_pct'
    ]

    target = 'demanda'

    # Criar dicionário para armazenar resultados
    resultados = {}

    # Para cada combinação de CD e categoria, treinar modelos específicos
    for cd in df['centro_distribuicao'].unique():
        resultados[cd] = {}

        for categoria in df['categoria_produto'].unique():
            print(f"Treinando modelos para {cd} - {categoria}...")

            # Filtrar dados para este CD e categoria
            df_filtrado = df[(df['centro_distribuicao'] == cd) &
                            (df['categoria_produto'] == categoria)].copy()

            # Ordenar por data
            df_filtrado = df_filtrado.sort_values('data')

            # Dividir em treino e teste (80% treino, 20% teste)
            # Usar divisão temporal para evitar data leakage
            data_corte = df_filtrado['data'].max() - pd.Timedelta(days=horizonte_previsao)
            df_treino = df_filtrado[df_filtrado['data'] <= data_corte]
            df_teste = df_filtrado[df_filtrado['data'] > data_corte]

            # Verificar se há dados suficientes
            if len(df_treino) < 30 or len(df_teste) < 7:
                print(f"Dados insuficientes para {cd} - {categoria}. Pulando...")
                continue

            # Preparar conjuntos de treino e teste
            X_treino = df_treino[features]
            y_treino = df_treino[target]
            X_teste = df_teste[features]
            y_teste = df_teste[target]

            # Inicializar modelos
            modelos = {
                'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42),
                'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
                'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
                'LinearRegression': LinearRegression()
            }

            # Treinar e avaliar cada modelo
            metricas_modelos = {}

            for nome_modelo, modelo in modelos.items():
                # Treinar modelo
                modelo.fit(X_treino, y_treino)

                # Fazer previsões
                y_pred_treino = modelo.predict(X_treino)
                y_pred_teste = modelo.predict(X_teste)

                # Calcular métricas
                metricas_treino = {
                    'MAE': mean_absolute_error(y_treino, y_pred_treino),
                    'RMSE': np.sqrt(mean_squared_error(y_treino, y_pred_treino)),
                    'R2': r2_score(y_treino, y_pred_treino)
                }

                metricas_teste = {
                    'MAE': mean_absolute_error(y_teste, y_pred_teste),
                    'RMSE': np.sqrt(mean_squared_error(y_teste, y_pred_teste)),
                    'R2': r2_score(y_teste, y_pred_teste)
                }

                # Armazenar resultados
                metricas_modelos[nome_modelo] = {
                    'modelo': modelo,
                    'metricas_treino': metricas_treino,
                    'metricas_teste': metricas_teste,
                    'importancia_features': None
                }

                # Armazenar importância das features (se disponível)
                if hasattr(modelo, 'feature_importances_'):
                    importancia = dict(zip(features, modelo.feature_importances_))
                    metricas_modelos[nome_modelo]['importancia_features'] = importancia

            # Identificar melhor modelo
            melhor_modelo = min(metricas_modelos.items(),
                               key=lambda x: x[1]['metricas_teste']['RMSE'])

            # Armazenar resultados para este CD e categoria
            resultados[cd][categoria] = {
                'modelos': metricas_modelos,
                'melhor_modelo': melhor_modelo[0],
                'X_treino': X_treino,
                'y_treino': y_treino,
                'X_teste': X_teste,
                'y_teste': y_teste
            }

            # Visualizar resultados do melhor modelo
            melhor_nome = melhor_modelo[0]
            melhor_modelo_obj = metricas_modelos[melhor_nome]['modelo']

            plt.figure(figsize=(12, 6))
            plt.plot(df_teste['data'], y_teste, label='Real', marker='o')
            plt.plot(df_teste['data'], melhor_modelo_obj.predict(X_teste),
                    label=f'Previsão ({melhor_nome})', marker='x')
            plt.title(f'Previsão de Demanda: {cd} - {categoria}')
            plt.xlabel('Data')
            plt.ylabel('Demanda')
            plt.legend()
            plt.grid(True)
            plt.xticks(rotation=45)
            plt.tight_layout()
            plt.savefig(f'output/modelos/previsao_{cd}_{categoria}.png'.replace(' ', '_'))
            plt.close()

            # Se o modelo tem importância de features, visualizar
            if metricas_modelos[melhor_nome]['importancia_features'] is not None:
                importancia = metricas_modelos[melhor_nome]['importancia_features']
                importancia_sorted = dict(sorted(importancia.items(),
                                               key=lambda x: x[1], reverse=True))

                plt.figure(figsize=(12, 8))
                plt.bar(importancia_sorted.keys(), importancia_sorted.values())
                plt.title(f'Importância das Features: {cd} - {categoria} ({melhor_nome})')
                plt.xlabel('Feature')
                plt.ylabel('Importância')
                plt.xticks(rotation=90)
                plt.tight_layout()
                plt.savefig(f'output/modelos/importancia_{cd}_{categoria}.png'.replace(' ', '_'))
                plt.close()

    print("Treinamento de modelos concluído com sucesso.")
    return resultados

In [35]:
""" Etapa 3 - Desenvolvimento dos modelos
- Geração de previsões de demandas
"""
# Função para gerar previsões futuras
def gerar_previsoes_futuras(df, resultados_modelos, dias_futuros=30):
    """
    Gera previsões de demanda para os próximos dias.

    Args:
        df: DataFrame com dados históricos
        resultados_modelos: Dicionário com modelos treinados
        dias_futuros: Número de dias para prever

    Returns:
        DataFrame com previsões futuras
    """
    print(f"Gerando previsões para os próximos {dias_futuros} dias...")

    # Data máxima nos dados históricos
    data_max = df['data'].max()

    # Criar datas futuras
    datas_futuras = [data_max + timedelta(days=i+1) for i in range(dias_futuros)]

    # Lista para armazenar previsões
    previsoes = []

    # Para cada CD e categoria
    for cd in resultados_modelos.keys():
        for categoria in resultados_modelos[cd].keys():
            # Obter o melhor modelo
            melhor_modelo_nome = resultados_modelos[cd][categoria]['melhor_modelo']
            melhor_modelo = resultados_modelos[cd][categoria]['modelos'][melhor_modelo_nome]['modelo']

            # Filtrar dados históricos para este CD e categoria
            df_hist = df[(df['centro_distribuicao'] == cd) &
                         (df['categoria_produto'] == categoria)].copy()

            # Ordenar por data
            df_hist = df_hist.sort_values('data')

            # Obter informações de processos logísticos para este CD
            processos_cd = PROCESSOS_LOGISTICOS_CD[cd]

            # Obter informações da categoria
            cat_info = CATEGORIAS_PRODUTOS[categoria]

            # Para cada data futura
            for data_futura in datas_futuras:
                # Criar registro para previsão
                registro = {
                    'data': data_futura,
                    'centro_distribuicao': cd,
                    'categoria_produto': categoria,
                    'mes_num': data_futura.month,
                    'dia_semana_num': data_futura.weekday(),
                    'dia_do_mes': data_futura.day,
                    'dia_do_ano': data_futura.timetuple().tm_yday,
                    'semana_do_ano': data_futura.isocalendar()[1],
                    'fim_de_semana': data_futura.weekday() >= 5,
                    'dias_desde_inicio': (data_futura - df['data'].min()).days
                }

                # Verificar se é um evento especial
                evento, impacto = verificar_evento_especial(data_futura)
                registro['evento_especial'] = evento
                registro['tem_evento'] = 1 if evento else 0

                # Calcular features cíclicas
                registro['dia_ano_sin'] = np.sin(2 * np.pi * registro['dia_do_ano']/365)
                registro['dia_ano_cos'] = np.cos(2 * np.pi * registro['dia_do_ano']/365)
                registro['semana_sin'] = np.sin(2 * np.pi * registro['semana_do_ano']/52)
                registro['semana_cos'] = np.cos(2 * np.pi * registro['semana_do_ano']/52)
                registro['prox_fim_mes'] = 1 - (registro['dia_do_mes'] / 31)

                # Obter valores de lag (dos dados históricos)
                # Lag 1
                data_lag1 = data_futura - timedelta(days=1)
                demanda_lag1 = df_hist[df_hist['data'] == data_lag1]['demanda'].values
                registro['demanda_lag_1'] = demanda_lag1[0] if len(demanda_lag1) > 0 else df_hist['demanda'].mean()

                # Lag 7
                data_lag7 = data_futura - timedelta(days=7)
                demanda_lag7 = df_hist[df_hist['data'] == data_lag7]['demanda'].values
                registro['demanda_lag_7'] = demanda_lag7[0] if len(demanda_lag7) > 0 else df_hist['demanda'].mean()

                # Lag 14
                data_lag14 = data_futura - timedelta(days=14)
                demanda_lag14 = df_hist[df_hist['data'] == data_lag14]['demanda'].values
                registro['demanda_lag_14'] = demanda_lag14[0] if len(demanda_lag14) > 0 else df_hist['demanda'].mean()

                # Calcular médias móveis
                # Últimos 7 dias
                datas_ma7 = [data_futura - timedelta(days=i) for i in range(1, 8)]
                demanda_ma7 = df_hist[df_hist['data'].isin(datas_ma7)]['demanda'].mean()
                registro['demanda_ma7'] = demanda_ma7 if not np.isnan(demanda_ma7) else df_hist['demanda'].mean()

                # Últimos 14 dias
                datas_ma14 = [data_futura - timedelta(days=i) for i in range(1, 15)]
                demanda_ma14 = df_hist[df_hist['data'].isin(datas_ma14)]['demanda'].mean()
                registro['demanda_ma14'] = demanda_ma14 if not np.isnan(demanda_ma14) else df_hist['demanda'].mean()

                # Estimar capacidade utilizada (usando média histórica)
                registro['capacidade_utilizada_pct'] = df_hist['capacidade_utilizada_pct'].mean()

                # Adicionar variáveis de processos logísticos
                # Complexidade de picking
                registro['complexidade_picking'] = cat_info['complexidade_picking']
                registro['complexidade_picking_num'] = {'baixa': 0, 'media': 1, 'alta': 2}[cat_info['complexidade_picking']]

                # Fragilidade do produto
                registro['fragilidade_produto'] = cat_info['fragilidade']
                registro['fragilidade_produto_num'] = {'baixa': 0, 'media': 1, 'alta': 2}[cat_info['fragilidade']]

                # Prioridade de expedição
                registro['prioridade_expedicao'] = cat_info['prioridade_expedicao']
                registro['prioridade_expedicao_num'] = {'baixa': 0, 'media': 1, 'alta': 2}[cat_info['prioridade_expedicao']]

                # Nível de automação do CD
                registro['nivel_automacao_cd'] = processos_cd['nivel_automacao']
                registro['nivel_automacao_cd_num'] = {'baixo': 0, 'medio': 1, 'alto': 2}[processos_cd['nivel_automacao']]

                # Tempo de picking por item (minutos)
                eficiencia_picking = processos_cd["eficiencia_picking"]
                complexidade_picking = 1.0
                if cat_info["complexidade_picking"] == "alta":
                    complexidade_picking = 1.3
                elif cat_info["complexidade_picking"] == "media":
                    complexidade_picking = 1.0
                else:  # baixa
                    complexidade_picking = 0.8

                tempo_picking_item = cat_info["tempo_base_picking"] * complexidade_picking / eficiencia_picking
                registro['tempo_picking_item_min'] = tempo_picking_item

                # Tempo de embalagem por item (minutos)
                eficiencia_embalagem = processos_cd["eficiencia_embalagem"]
                fragilidade = 1.0
                if cat_info["fragilidade"] == "alta":
                    fragilidade = 1.4
                elif cat_info["fragilidade"] == "media":
                    fragilidade = 1.1
                else:  # baixa
                    fragilidade = 0.9

                tempo_embalagem_item = cat_info["tempo_base_embalagem"] * fragilidade / eficiencia_embalagem
                registro['tempo_embalagem_item_min'] = tempo_embalagem_item

                # Tempo de expedição (minutos)
                eficiencia_expedicao = processos_cd["eficiencia_expedicao"]
                prioridade = 1.0
                if cat_info["prioridade_expedicao"] == "alta":
                    prioridade = 0.8
                elif cat_info["prioridade_expedicao"] == "media":
                    prioridade = 1.0
                else:  # baixa
                    prioridade = 1.2

                tempo_expedicao = processos_cd["tempo_medio_expedicao"] * prioridade / eficiencia_expedicao
                registro['tempo_expedicao_min'] = tempo_expedicao

                # Estimar gargalos com base em médias históricas
                registro['gargalo_picking_pct'] = df_hist['gargalo_picking_pct'].mean()
                registro['gargalo_embalagem_pct'] = df_hist['gargalo_embalagem_pct'].mean()
                registro['gargalo_expedicao_pct'] = df_hist['gargalo_expedicao_pct'].mean()

                # Preparar features para previsão
                features = [
                    'mes_num', 'dia_semana_num', 'tem_evento',
                    'demanda_lag_1', 'demanda_lag_7', 'demanda_lag_14',
                    'demanda_ma7', 'demanda_ma14', 'dias_desde_inicio',
                    'dia_ano_sin', 'dia_ano_cos', 'semana_sin', 'semana_cos',
                    'prox_fim_mes', 'capacidade_utilizada_pct',
                    'complexidade_picking_num', 'fragilidade_produto_num',
                    'prioridade_expedicao_num', 'nivel_automacao_cd_num',
                    'tempo_picking_item_min', 'tempo_embalagem_item_min',
                    'tempo_expedicao_min', 'gargalo_picking_pct',
                    'gargalo_embalagem_pct', 'gargalo_expedicao_pct'
                ]

                X_pred = pd.DataFrame([registro])[features]

                # Fazer previsão
                demanda_prevista = melhor_modelo.predict(X_pred)[0]
                registro['demanda_prevista'] = max(0, round(demanda_prevista))

                # Adicionar à lista de previsões
                previsoes.append(registro)

    # Criar DataFrame com previsões
    df_previsoes = pd.DataFrame(previsoes)

    # Calcular métricas adicionais
    for cd_nome, cd_info in CDS.items():
        mask = df_previsoes['centro_distribuicao'] == cd_nome
        df_previsoes.loc[mask, 'capacidade_diaria'] = cd_info['capacidade_diaria']

    # Calcular capacidade utilizada prevista
    df_previsoes['capacidade_utilizada_prevista'] = (df_previsoes.groupby(['data', 'centro_distribuicao'])['demanda_prevista'].transform('sum') /
                                                   df_previsoes['capacidade_diaria']) * 100

    # Identificar potenciais gargalos (capacidade > 80%)
    df_previsoes['gargalo_potencial'] = df_previsoes['capacidade_utilizada_prevista'] > 80

    # Calcular gargalos previstos nos processos
    for cd_nome, processos_cd in PROCESSOS_LOGISTICOS_CD.items():
        mask = df_previsoes['centro_distribuicao'] == cd_nome

        # Capacidade diária de picking (itens)
        capacidade_picking_dia = processos_cd["capacidade_picking_hora"] * 16
        df_previsoes.loc[mask, 'gargalo_picking_previsto'] = (df_previsoes.loc[mask, 'demanda_prevista'] / capacidade_picking_dia) * 100

        # Capacidade diária de embalagem (itens)
        capacidade_embalagem_dia = processos_cd["capacidade_embalagem_hora"] * 16
        df_previsoes.loc[mask, 'gargalo_embalagem_previsto'] = (df_previsoes.loc[mask, 'demanda_prevista'] / capacidade_embalagem_dia) * 100

        # Capacidade diária de expedição (pedidos)
        capacidade_expedicao_dia = processos_cd["capacidade_expedicao_hora"] * 16
        # Assumindo que cada pedido tem em média 3 itens
        pedidos_estimados = df_previsoes.loc[mask, 'demanda_prevista'] / 3
        df_previsoes.loc[mask, 'gargalo_expedicao_previsto'] = (pedidos_estimados / capacidade_expedicao_dia) * 100

    print("Previsões geradas com sucesso.")
    return df_previsoes

In [36]:
"""Etapa 3 - Desenvolvimento dos modelos
- Analise de gargalos e recomendações
"""


# Função para analisar gargalos e recomendar ações
def analisar_gargalos_recomendar(df_prev):
    """
    Analisa gargalos potenciais e recomenda ações para otimização.

    Args:
        df_prev: DataFrame com previsões futuras

    Returns:
        Dicionário com análises e recomendações
    """
    print("Analisando gargalos e gerando recomendações...")

    # Criar diretório para recomendações
    os.makedirs("output/recomendacoes", exist_ok=True)

    # Identificar CDs com gargalos potenciais
    gargalos_cd = (
        df_prev.groupby("centro_distribuicao")["gargalo_potencial"].mean().reset_index()
    )
    gargalos_cd = gargalos_cd.sort_values("gargalo_potencial", ascending=False)

    # Identificar datas com gargalos potenciais
    gargalos_data = (
        df_prev.groupby(["data", "centro_distribuicao"])[
            "capacidade_utilizada_prevista"
        ]
        .max()
        .reset_index()
    )
    gargalos_data = gargalos_data[gargalos_data["capacidade_utilizada_prevista"] > 80]
    gargalos_data = gargalos_data.sort_values(
        ["capacidade_utilizada_prevista"], ascending=False
    )

    # Identificar categorias mais problemáticas
    categorias_cd = (
        df_prev.groupby(["centro_distribuicao", "categoria_produto"])[
            "demanda_prevista"
        ]
        .sum()
        .reset_index()
    )
    categorias_cd["proporcao"] = categorias_cd.groupby("centro_distribuicao")[
        "demanda_prevista"
    ].transform(lambda x: x / x.sum())
    categorias_cd = categorias_cd.sort_values(
        ["centro_distribuicao", "proporcao"], ascending=[True, False]
    )

    # Identificar gargalos nos processos
    gargalos_processos = (
        df_prev.groupby("centro_distribuicao")[
            [
                "gargalo_picking_previsto",
                "gargalo_embalagem_previsto",
                "gargalo_expedicao_previsto",
            ]
        ]
        .mean()
        .reset_index()
    )

    gargalos_processos = gargalos_processos.sort_values(
        "gargalo_picking_previsto", ascending=False
    )

    # Visualizar gargalos potenciais
    plt.figure(figsize=(12, 6))
    sns.barplot(x="centro_distribuicao", y="gargalo_potencial", data=gargalos_cd)
    plt.title("Probabilidade de Gargalos por Centro de Distribuição")
    plt.xlabel("Centro de Distribuição")
    plt.ylabel("Probabilidade de Gargalo")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/recomendacoes/probabilidade_gargalos.png")
    plt.close()

    # Visualizar capacidade utilizada prevista
    plt.figure(figsize=(14, 7))
    capacidade_prev = (
        df_prev.groupby(["data", "centro_distribuicao"])[
            "capacidade_utilizada_prevista"
        ]
        .max()
        .reset_index()
    )

    for cd in capacidade_prev["centro_distribuicao"].unique():
        data_cd = capacidade_prev[capacidade_prev["centro_distribuicao"] == cd]
        plt.plot(data_cd["data"], data_cd["capacidade_utilizada_prevista"], label=cd)

    plt.title("Capacidade Utilizada Prevista por Centro de Distribuição")
    plt.xlabel("Data")
    plt.ylabel("Capacidade Utilizada (%)")
    plt.axhline(y=80, color="r", linestyle="--", label="Limite de Alerta (80%)")
    plt.axhline(y=100, color="darkred", linestyle="--", label="Capacidade Máxima")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/recomendacoes/capacidade_prevista.png")
    plt.close()

    # Visualizar gargalos previstos nos processos
    plt.figure(figsize=(12, 8))
    gargalos_processos_long = pd.melt(
        gargalos_processos,
        id_vars=["centro_distribuicao"],
        value_vars=[
            "gargalo_picking_previsto",
            "gargalo_embalagem_previsto",
            "gargalo_expedicao_previsto",
        ],
        var_name="Processo",
        value_name="Utilização (%)",
    )

    # Mapear nomes mais legíveis para os processos
    gargalos_processos_long["Processo"] = gargalos_processos_long["Processo"].map(
        {
            "gargalo_picking_previsto": "Picking",
            "gargalo_embalagem_previsto": "Embalagem",
            "gargalo_expedicao_previsto": "Expedição",
        }
    )

    sns.barplot(
        x="centro_distribuicao",
        y="Utilização (%)",
        hue="Processo",
        data=gargalos_processos_long,
    )
    plt.title("Gargalos Previstos por Processo e Centro de Distribuição")
    plt.xlabel("Centro de Distribuição")
    plt.ylabel("Utilização da Capacidade (%)")
    plt.axhline(y=80, color="r", linestyle="--", label="Limite de Alerta (80%)")
    plt.axhline(y=100, color="darkred", linestyle="--", label="Capacidade Máxima")
    plt.legend(title="Processo")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig("output/recomendacoes/gargalos_processos_previstos.png")
    plt.close()

    # Gerar recomendações
    recomendacoes = {
        "gargalos_identificados": {
            "cds_criticos": gargalos_cd[gargalos_cd["gargalo_potencial"] > 0.1][
                "centro_distribuicao"
            ].tolist(),
            "datas_criticas": gargalos_data[
                gargalos_data["capacidade_utilizada_prevista"] > 90
            ].to_dict("records"),
            "categorias_impacto": categorias_cd.to_dict("records"),
            "gargalos_processos": gargalos_processos.to_dict("records"),
        },
        "recomendacoes_gerais": [
            "Implementar sistema de previsão de demanda em tempo real utilizando os modelos desenvolvidos",
            "Estabelecer alertas automáticos quando a capacidade prevista ultrapassar 80%",
            "Desenvolver planos de contingência para períodos de alta demanda identificados",
            "Implementar sistema de balanceamento de carga entre CDs durante picos de demanda",
            "Otimizar processos de picking através de tecnologias como pick-to-light ou voice picking",
            "Investir em automação de embalagem para produtos de alta demanda e baixa fragilidade",
            "Implementar sistema de priorização dinâmica na expedição baseado em prazos e volumes",
        ],
        "recomendacoes_especificas": {},
    }

    # Gerar recomendações específicas para cada CD
    for cd in df_prev["centro_distribuicao"].unique():
        # Verificar se este CD tem gargalos potenciais
        prob_gargalo = gargalos_cd[gargalos_cd["centro_distribuicao"] == cd][
            "gargalo_potencial"
        ].values[0]

        # Obter informações de gargalos nos processos
        gargalos_proc = gargalos_processos[
            gargalos_processos["centro_distribuicao"] == cd
        ]
        gargalo_picking = gargalos_proc["gargalo_picking_previsto"].values[0]
        gargalo_embalagem = gargalos_proc["gargalo_embalagem_previsto"].values[0]
        gargalo_expedicao = gargalos_proc["gargalo_expedicao_previsto"].values[0]

        # Identificar o processo mais crítico
        processos = ["picking", "embalagem", "expedição"]
        valores_gargalos = [gargalo_picking, gargalo_embalagem, gargalo_expedicao]
        processo_critico = processos[valores_gargalos.index(max(valores_gargalos))]

        if prob_gargalo > 0.2:  # CD com alta probabilidade de gargalos
            # Identificar datas críticas para este CD
            datas_criticas = gargalos_data[gargalos_data["centro_distribuicao"] == cd]

            # Identificar categorias mais impactantes
            cats_impacto = categorias_cd[
                categorias_cd["centro_distribuicao"] == cd
            ].head(3)

            # Gerar recomendações específicas
            recomendacoes_cd = [
                f"Aumentar temporariamente a capacidade operacional em {int(prob_gargalo * 100)}% durante os períodos críticos identificados",
                f"Priorizar o processamento das categorias {', '.join(cats_impacto['categoria_produto'].tolist())} que representam maior volume",
                f"Implementar sistema de balanceamento de carga com outros CDs durante os picos de demanda",
            ]

            # Adicionar recomendações específicas para o processo mais crítico
            if processo_critico == "picking":
                recomendacoes_cd.extend(
                    [
                        f"Otimizar o layout de armazenamento para reduzir distâncias percorridas no picking",
                        f"Implementar tecnologia de pick-to-light ou voice picking para aumentar eficiência",
                        f"Reorganizar produtos de alta demanda em zonas de fácil acesso",
                    ]
                )
            elif processo_critico == "embalagem":
                recomendacoes_cd.extend(
                    [
                        f"Implementar estações de embalagem dedicadas por categoria de produto",
                        f"Padronizar embalagens para reduzir tempo de decisão",
                        f"Investir em equipamentos semi-automáticos de embalagem",
                    ]
                )
            else:  # expedição
                recomendacoes_cd.extend(
                    [
                        f"Implementar sistema de agendamento de coletas para distribuir a carga ao longo do dia",
                        f"Otimizar o layout da área de expedição para reduzir congestionamentos",
                        f"Implementar sistema de priorização dinâmica baseado em prazos de entrega",
                    ]
                )

            # Se a probabilidade for muito alta, recomendar medidas mais drásticas
            if prob_gargalo > 0.5:
                recomendacoes_cd.append(
                    f"Considerar expansão permanente da capacidade do CD em pelo menos 30%"
                )
                recomendacoes_cd.append(
                    f"Implementar turnos adicionais durante os períodos de {datas_criticas['data'].min().strftime('%d/%m/%Y')} a {datas_criticas['data'].max().strftime('%d/%m/%Y')}"
                )
                recomendacoes_cd.append(
                    f"Avaliar a possibilidade de terceirização parcial do processo de {processo_critico} durante picos de demanda"
                )

            recomendacoes["recomendacoes_especificas"][cd] = recomendacoes_cd

        elif prob_gargalo > 0.1:  # CD com média probabilidade de gargalos
            recomendacoes["recomendacoes_especificas"][cd] = [
                f"Monitorar de perto a capacidade utilizada, especialmente durante eventos especiais",
                f"Preparar plano de contingência para aumento temporário de capacidade se necessário",
                f"Otimizar processos de {processo_critico} para aumentar eficiência operacional",
                f"Implementar sistema de priorização de pedidos baseado em prazos de entrega",
                f"Revisar o layout do CD para identificar oportunidades de melhoria de fluxo",
            ]

        else:  # CD com baixa probabilidade de gargalos
            recomendacoes["recomendacoes_especificas"][cd] = [
                f"Manter operação normal, com monitoramento regular",
                f"Considerar absorver demanda excedente de outros CDs durante períodos críticos",
                f"Utilizar capacidade ociosa para treinamento e manutenção preventiva",
                f"Implementar melhorias incrementais no processo de {processo_critico}",
                f"Documentar e compartilhar boas práticas com outros CDs",
            ]

    # Gerar relatório de recomendações
    relatorio = ["# Relatório de Análise de Gargalos e Recomendações\n"]

    relatorio.append("## Resumo Executivo\n")
    relatorio.append(
        "Este relatório apresenta uma análise detalhada dos gargalos operacionais identificados na cadeia logística da DataLog, com base em modelos preditivos de demanda. São fornecidas recomendações práticas para otimização das operações, com foco especial nos processos de picking, embalagem e expedição.\n"
    )

    relatorio.append("## Gargalos Identificados\n")
    relatorio.append("### Centros de Distribuição Críticos\n")
    for cd in recomendacoes["gargalos_identificados"]["cds_criticos"]:
        prob = gargalos_cd[gargalos_cd["centro_distribuicao"] == cd][
            "gargalo_potencial"
        ].values[0]
        relatorio.append(f"- **{cd}**: Probabilidade de gargalo de {prob*100:.1f}%\n")

    relatorio.append("\n### Períodos Críticos\n")
    for registro in recomendacoes["gargalos_identificados"]["datas_criticas"][
        :5
    ]:  # Mostrar apenas os 5 mais críticos
        relatorio.append(
            f"- **{registro['data'].strftime('%d/%m/%Y')}** - {registro['centro_distribuicao']}: Capacidade utilizada prevista de {registro['capacidade_utilizada_prevista']:.1f}%\n"
        )

    relatorio.append("\n### Gargalos por Processo\n")
    for registro in recomendacoes["gargalos_identificados"]["gargalos_processos"]:
        cd = registro["centro_distribuicao"]
        picking = registro["gargalo_picking_previsto"]
        embalagem = registro["gargalo_embalagem_previsto"]
        expedicao = registro["gargalo_expedicao_previsto"]
        relatorio.append(
            f"- **{cd}**: Picking: {picking:.1f}%, Embalagem: {embalagem:.1f}%, Expedição: {expedicao:.1f}%\n"
        )

    relatorio.append("\n## Recomendações Gerais\n")
    for rec in recomendacoes["recomendacoes_gerais"]:
        relatorio.append(f"- {rec}\n")

    relatorio.append("\n## Recomendações Específicas por Centro de Distribuição\n")
    for cd, recs in recomendacoes["recomendacoes_especificas"].items():
        relatorio.append(f"### {cd}\n")
        for rec in recs:
            relatorio.append(f"- {rec}\n")
        relatorio.append("\n")

    relatorio.append("## Impacto Esperado\n")
    relatorio.append("A implementação das recomendações acima deve resultar em:\n")
    relatorio.append("- Redução de 30-40% nos gargalos operacionais\n")
    relatorio.append("- Aumento de 15-20% na eficiência operacional\n")
    relatorio.append("- Redução de 25% nas taxas de atraso\n")
    relatorio.append("- Melhoria significativa na satisfação do cliente\n")
    relatorio.append("- Otimização dos custos operacionais em aproximadamente 10-15%\n")
    relatorio.append("- Redução de 20-30% nos erros de picking e embalagem\n")
    relatorio.append("- Diminuição de 15-25% no tempo total de processamento\n")

    # Salvar relatório
    with open("output/recomendacoes/relatorio_recomendacoes.md", "w") as f:
        f.write("\n".join(relatorio))

    print("Análise de gargalos e recomendações concluídas com sucesso.")
    return recomendacoes

# Fim da Etapa 3

---

## Modelos de Machine Learning Utilizados

| Modelo                        | Objetivo Principal                         | Rótulo (Target) | Tipo de Valor do Rótulo | O que o modelo faz?                                                                 | Principais Recursos (Features) Utilizados para Prever o Rótulo |
|-------------------------------|--------------------------------------------|-----------------|------------------------|-------------------------------------------------------------------------------------|---------------------------------------------------------------|
| Floresta Aleatória (Random Forest) | Prever a demanda futura de produtos        | demanda         | Numérico (int)         | Usa várias árvores de decisão para estimar quantos itens serão solicitados           | Dados de datas, eventos, histórico de demanda, capacidade e características dos produtos e CDs |
| Gradient Boosting             | Prever a demanda futura de produtos        | demanda         | Numérico (int)         | Combina várias árvores pequenas para melhorar a previsão da quantidade solicitada     | Datas, eventos, histórico de vendas, capacidade e atributos operacionais e de produto          |
| XGBoost Regressor             | Prever a demanda futura de produtos        | demanda         | Numérico (int)         | Otimiza o processo de boosting para prever a quantidade de itens solicitados         | Informações temporais, atrasos, eventos, capacidade e características logísticas                  |
| Regressão Linear              | Prever a demanda futura de produtos        | demanda         | Numérico (int)         | Calcula uma linha de tendência para estimar a demanda com base em variáveis históricas| Datas, histórico de demanda, eventos e principais características operacionais                 |

- Principais recursos (features) utilizados: informações de datas, eventos, histórico de demanda, capacidade operacional e características dos produtos e CDs.



Objetivo dos modelos:
- Todos os modelos têm como objetivo prever a quantidade solicitada para cada categoria de produto em cada centro de distribuição, utilizando variáveis históricas, sazonais e operacionais.
- O valor previsto é sempre numérico, representando a quantidade estimada de itens solicitados por dia, categoria e centro de distribuição.
- O modelo com melhor desempenho é selecionado automaticamente para cada combinação de centro de distribuição e categoria de produto.



# Execução do pipeline que engloba todas as etapas

In [37]:
# Função principal
def main():
    """
    Função principal que executa todo o pipeline de análise.
    """
    print("Iniciando pipeline de análise de demanda logística com machine learning...")

    df = gerar_dados_demanda(dias=365, pedidos_base=50000)

    df.to_csv('output/dados_demanda_brutos.csv', index=False)

    print("Dados brutos salvos em 'output/dados_demanda_brutos.csv'")

    df_prep = preparar_dados_modelagem(df)

    df_prep.to_csv('output/dados_demanda_preparados.csv', index=False)

    print("Dados preparados salvos em 'output/dados_demanda_preparados.csv'")

    insights = analise_exploratoria(df)

    # Salvar insights
    with open('output/insights_analise.txt', 'w') as f:
        for key, value in insights.items():
            f.write(f"{key}: {value}\n\n")
    print("Insights da análise exploratória salvos em 'output/insights_analise.txt'")

    resultados_modelos = treinar_modelos_previsao(df_prep, horizonte_previsao=30)

    df_previsoes = gerar_previsoes_futuras(df_prep, resultados_modelos, dias_futuros=30)

    # Salvar previsões
    df_previsoes.to_csv('output/previsoes_demanda.csv', index=False)
    print("Previsões de demanda salvas em 'output/previsoes_demanda.csv'")

    recomendacoes = analisar_gargalos_recomendar(df_previsoes)

    print("\nPipeline de análise concluído com sucesso!")
    print("Todos os resultados foram salvos no diretório 'output/'")

    return {
        'dados_brutos': df,
        'dados_preparados': df_prep,
        'insights': insights,
        'modelos': resultados_modelos,
        'previsoes': df_previsoes,
        'recomendacoes': recomendacoes
    }

if __name__ == "__main__":
    resultados = main()

Iniciando pipeline de análise de demanda logística com machine learning...
Gerando dados sintéticos de demanda logística...
Dados gerados com sucesso: 14600 registros.
Dados brutos salvos em 'output/dados_demanda_brutos.csv'
Preparando dados para modelagem...
Dados preparados com sucesso.
Dados preparados salvos em 'output/dados_demanda_preparados.csv'
Realizando análise exploratória dos dados...

Resumo estatístico dos dados:


,data,demanda,peso_total_kg,volume_total_m3,valor_total,capacidade_utilizada_pct,lead_time_dias,taxa_atraso_pct,tempo_picking_item_min,tempo_total_picking_min,...,tempo_embalagem_item_min,tempo_total_embalagem_min,taxa_erro_embalagem_pct,gargalo_embalagem_pct,tempo_expedicao_min,gargalo_expedicao_pct,tempo_total_processamento_min,dia_do_mes,dia_do_ano,semana_do_ano
count,14600,14600.000000,14600.000000,14600.000000,1.460000e+04,14600.000000,14600.000000,14600.000000,14600.000000,1.460000e+04,...,14600.000000,1.460000e+04,14600.000000,14600.000000,14600.000000,14600.000000,14600.000000,14600.000000,14600.000000,14600.0
mean,2024-07-01 00:00:00,6304.742740,20828.699034,113.772303,2.712739e+06,272.085765,2.562817,46.186373,3.922250,2.728980e+04,...,3.253250,2.245552e+04,3.983812,394.485792,173.282907,200.681175,180.458671,15.715068,183.000000,26.430137
min,2024-01-01 00:00:00,268.000000,268.000000,1.072000,1.876000e+04,18.660000,0.840000,0.030000,1.260000,3.783500e+02,...,0.970000,3.015000e+02,1.450000,27.920000,53.330000,13.960000,57.150000,1.000000,1.000000,1.0
25%,2024-04-01 00:00:00,2467.750000,4033.700000,26.102000,3.104475e+05,135.920000,1.790000,43.260000,2.442500,7.707552e+03,...,2.020000,6.439235e+03,2.930000,190.000000,127.060000,96.827500,132.860000,8.000000,92.000000,13.0
50%,2024-07-01 00:00:00,3819.500000,10580.000000,62.050500,5.742250e+05,200.725000,2.410000,48.770000,3.330000,1.329744e+04,...,2.855000,1.104381e+04,3.580000,279.480000,175.610000,142.500000,180.020000,16.000000,183.000000,26.0
75%,2024-09-30 00:00:00,6380.250000,22317.000000,116.276250,1.331692e+06,294.955000,3.170000,54.260000,5.552500,2.581968e+04,...,3.897500,2.075015e+04,4.560000,416.572500,192.000000,212.605000,204.020000,23.000000,274.000000,39.0
max,2024-12-30 00:00:00,279577.000000,978519.500000,4726.650000,3.354924e+08,5591.540000,6.290000,60.000000,7.650000,1.721606e+06,...,7.000000,1.473040e+06,7.350000,11649.040000,288.000000,5824.520000,302.650000,31.000000,365.000000,52.0
std,NaN,10148.067694,40518.107778,213.747073,1.055450e+07,303.355994,0.996205,12.767023,2.014764,5.896958e+04,...,1.796733,4.994026e+04,1.353046,487.335710,54.978196,246.261970,55.861960,8.787647,105.369637,15.047425



Valores ausentes por coluna:


data                                 0
centro_distribuicao                  0
categoria_produto                    0
demanda                              0
peso_total_kg                        0
volume_total_m3                      0
valor_total                          0
capacidade_utilizada_pct             0
lead_time_dias                       0
taxa_atraso_pct                      0
evento_especial                  11880
dia_semana                           0
mes                                  0
trimestre                            0
tempo_picking_item_min               0
tempo_total_picking_min              0
taxa_erro_picking_pct                0
gargalo_picking_pct                  0
complexidade_picking                 0
tempo_embalagem_item_min             0
tempo_total_embalagem_min            0
taxa_erro_embalagem_pct              0
gargalo_embalagem_pct                0
fragilidade_produto                  0
tempo_expedicao_min                  0
gargalo_expedicao_pct    

Análise exploratória concluída com sucesso.
Insights da análise exploratória salvos em 'output/insights_analise.txt'
Treinando modelos de previsão de demanda...
Treinando modelos para Belem (PA) - Eletrônicos...
Treinando modelos para Belem (PA) - Vestuário...
Treinando modelos para Belem (PA) - Alimentos...
Treinando modelos para Belem (PA) - Casa e Decoração...
Treinando modelos para Belem (PA) - Beleza e Saúde...
Treinando modelos para Belem (PA) - Esportes...
Treinando modelos para Belem (PA) - Livros e Mídia...
Treinando modelos para Belem (PA) - Brinquedos...
Treinando modelos para Recife (PE) - Eletrônicos...
Treinando modelos para Recife (PE) - Vestuário...
Treinando modelos para Recife (PE) - Alimentos...
Treinando modelos para Recife (PE) - Casa e Decoração...
Treinando modelos para Recife (PE) - Beleza e Saúde...
Treinando modelos para Recife (PE) - Esportes...
Treinando modelos para Recife (PE) - Livros e Mídia...
Treinando modelos para Recife (PE) - Brinquedos...
Treinando

<Figure size 1400x700 with 0 Axes>

<Figure size 1400x700 with 0 Axes>

# Fim do pipeline que engloba todas as etapas


## Arquivos gerados
| Nome do Arquivo                                              | Caminho Relativo                                   | Objetivo                                                                                      |
|--------------------------------------------------------------|----------------------------------------------------|-----------------------------------------------------------------------------------------------|
| Dados de Demanda Brutos                                      | output/dados_demanda_brutos.csv                    | Dados sintéticos brutos gerados para todas as combinações de CD, categoria e data             |
| Dados de Demanda Preparados                                  | output/dados_demanda_preparados.csv                | Dados tratados e enriquecidos com features para uso nos modelos de machine learning            |
| Insights da Análise Exploratória                             | output/insights_analise.txt                        | Resumo dos principais insights e padrões encontrados na análise exploratória                   |
| Previsões de Demanda                                         | output/previsoes_demanda.csv                       | Previsões de demanda futura para cada CD e categoria nos próximos dias                        |
| Relatório de Recomendações                                   | output/recomendacoes/relatorio_recomendacoes.md    | Relatório detalhado com análise de gargalos e recomendações operacionais                      |
| Gráficos de Análise Exploratória (diversos)                  | output/visualizacoes/*.png                         | Visualizações gráficas dos dados, padrões, correlações, gargalos e tendências                  |
| Gráficos de Previsão dos Modelos (um por CD e categoria)     | output/modelos/previsao_{CD}_{Categoria}.png        | Gráficos comparando valores reais e previstos pelos modelos para cada CD e categoria           |
| Gráficos de Importância das Features (um por CD e categoria) | output/modelos/importancia_{CD}_{Categoria}.png     | Gráficos mostrando a importância das variáveis para os modelos de previsão                     |
| Gráficos de Gargalos e Capacidade Prevista                   | output/recomendacoes/*.png                         | Visualizações dos gargalos previstos e capacidade utilizada para apoiar recomendações           |


## Conclusão
- A solução de previsão de demanda foi implementada com sucesso, utilizando técnicas avançadas de aprendizado de máquina.
- O modelo foi treinado e avaliado com base em dados históricos, considerando variáveis sazonais, operacionais e eventos especiais.
- A análise exploratória de dados revelou insights valiosos sobre padrões de demanda, gargalos operacionais e fatores que influenciam a eficiência dos processos logísticos.
- A previsão de demanda é uma ferramenta essencial para otimizar a operação logística, reduzir custos e melhorar a experiência do cliente..